In [1]:
import sys
import pathlib
src_path = pathlib.Path().absolute().parent.parent / "src"
sys.path.append(str(src_path))

import data_preprocessing as dp
import model_evaluation as me

In [2]:
import pandas as pd
import numpy as np
import random

from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelBinarizer, LabelEncoder

import re
from nltk import pos_tag
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer 

import seaborn as sns
import matplotlib.pyplot as plt


In [3]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding, Flatten
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dropout, Activation, Bidirectional
from keras.models import Sequential
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence

In [42]:
# df = pd.read_csv('../../data/judge-1377884607_tweet_product_company.csv', encoding = 'latin1')
# df.columns = ['text', 'product', 'emotion']
# df = df[df['emotion'] != 'I can\'t tell']
# df['product'].fillna('No Product', inplace = True)
# df.dropna(inplace = True)

# get data using andrew's clean data functions

In [6]:
cleaned_df = dp.df_clean(lem=False)

In [7]:
cleaned_df.head(5)

,emotion,txt_cleaned
0,0,product_target hrs tweeting riseaustin dead ne...
1,2,know awesome ipadiphone product_target youll l...
2,2,wait product_target also sale sxsw
3,0,hope years festival isnt crashy years iphone p...
4,2,great stuff fri sxsw marissa mayer product_tar...


In [8]:
y = pd.get_dummies(cleaned_df['emotion'])
y

,0,1,2
0,1,0,0
1,0,0,1
2,0,0,1
3,1,0,0
4,0,0,1
...,...,...,...
9077,0,0,1
9079,0,0,1
9080,1,0,0
9085,0,0,1


In [10]:
X = cleaned_df['txt_cleaned']
X

0       product_target hrs tweeting riseaustin dead ne...
1       know awesome ipadiphone product_target youll l...
2                      wait product_target also sale sxsw
3       hope years festival isnt crashy years iphone p...
4       great stuff fri sxsw marissa mayer product_tar...
                              ...                        
9077    pr guy convinced switch back product_target gr...
9079    quotpapyrussort like ipadquot nice lol sxsw la...
9080    diller says google tv quotmight run playstatio...
9085    ive always used camera iphone bc image stabili...
9088                       product_target everywhere sxsw
Name: txt_cleaned, Length: 3282, dtype: object

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)
X_tr, X_v, y_tr, y_v = train_test_split(X_train, y_train, test_size=.1)

# tokenize tweets

In [12]:
X_train_list = list(X_train)
X_test_list = list(X_test)
X_tr_list = list(X_tr)
X_v_list = list(X_v)

X_train_corpus = []
X_tr_corpus = []
for tweet in X_train_list:
    X_train_corpus.extend(tweet.split(' '))
for tweet in X_tr_list:
    X_tr_corpus.extend(tweet.split(' '))
    
X_train_unique = len(set(X_train_corpus))
X_tr_unique = len(set(X_tr_corpus))

In [20]:
sequence_length = 100

partial_tokenizer = text.Tokenizer(num_words=X_tr_unique)
full_tokenizer = text.Tokenizer(num_words=X_train_unique)

partial_tokenizer.fit_on_texts(X_tr_list)
full_tokenizer.fit_on_texts(X_train_list)

X_tr_tokenized = partial_tokenizer.texts_to_sequences(X_tr_list)
X_v_tokenized = partial_tokenizer.texts_to_sequences(X_v_list)
X_train_tokenized= full_tokenizer.texts_to_sequences(X_train_list)
X_test_tokenized= full_tokenizer.texts_to_sequences(X_test_list)

X_tr_tokens = sequence.pad_sequences(X_tr_tokenized, maxlen=sequence_length)
X_v_tokens = sequence.pad_sequences(X_v_tokenized, maxlen=sequence_length)
X_train_tokens = sequence.pad_sequences(X_train_tokenized, maxlen=sequence_length)
X_test_tokens = sequence.pad_sequences(X_test_tokenized, maxlen=sequence_length)

#### ^^^ modified for train_test_split variables up to here

# construct CNN with embedding layer

In [63]:
model = Sequential()

embedding_size = 128
model.add(Embedding(input_dim=len(set(corpus)), output_dim=embedding_size, input_length=100))
model.add(Conv1D(filters=32, kernel_size=4, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(units=30, activation='tanh'))
model.add(Dense(units=3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit model on tokenized data

In [64]:
model.fit(X_t, y, epochs=7, batch_size=32, validation_split=0.1)

Epoch 1/7
252/252 [==============================] - 9s 35ms/step - loss: 0.8114 - accuracy: 0.6124 - val_loss: 0.7427 - val_accuracy: 0.6387
Epoch 2/7
252/252 [==============================] - 8s 33ms/step - loss: 0.5957 - accuracy: 0.7457 - val_loss: 0.7099 - val_accuracy: 0.6633
Epoch 3/7
252/252 [==============================] - 8s 33ms/step - loss: 0.4050 - accuracy: 0.8401 - val_loss: 0.7726 - val_accuracy: 0.6465
Epoch 4/7
252/252 [==============================] - 9s 34ms/step - loss: 0.3069 - accuracy: 0.8834 - val_loss: 0.8015 - val_accuracy: 0.6622
Epoch 5/7
252/252 [==============================] - 8s 33ms/step - loss: 0.2518 - accuracy: 0.9021 - val_loss: 0.8266 - val_accuracy: 0.6723
Epoch 6/7
252/252 [==============================] - 9s 36ms/step - loss: 0.2202 - accuracy: 0.9126 - val_loss: 0.9046 - val_accuracy: 0.6667
Epoch 7/7
252/252 [==============================] - 9s 34ms/step - loss: 0.2083 - accuracy: 0.9159 - val_loss: 0.9278 - val_accuracy: 0.6566
